# CITS3401 Project 1
### Kean Scott - 23850149

## Introduction

In Australia, motor vehicle accidents make up a significant proportion of potentially preventable 

The following python notebook contains all the steps for the analysis of the Australian Road Deaths Database (**ARDD**), also providing the steps taken to build a daata warehouse suitable for ongoing observation of key metrics and statistics. These key metrics will be defined to answer particular questions reguarding road deaths in Australia, in an effort to uncover underlying trends and correlations in these fatalities.  

Here are the original sources of my data:

- [Fatal crashes (updated Feb 2025)](https://catalogue.data.infrastructure.gov.au/dataset/australian-road-deaths-database/resource/457dbf98-419e-4f1e-a45f-4d568ff0ff69?inner_span=True)
- [Fatalities (updated Feb 2025)](https://catalogue.data.infrastructure.gov.au/dataset/australian-road-deaths-database/resource/80091814-9a39-444c-a329-b27561d8fcc6?inner_span=True)

## Step 1: Initial Data Exploration - Identifying the questions to be asked of our data warehouse, and defining it's dimensions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
fatal_crashes = pd.read_csv("./data/fatal_crashes_feb2025.csv")
fatalities = pd.read_csv("./data/fatalities_feb2025.csv")

In [3]:
fatalities.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,Road User,Gender,Age,National Remoteness Areas 2021,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period
0,120161098270,NSW,4,2016,Monday,15:29:00,Multiple,No,No,No,100,Driver,Male,76,Inner Regional Australia,New England and North West,Tamworth Regional,Arterial Road,No,No
1,120161097596,NSW,4,2016,Tuesday,16:40:00,Multiple,No,No,Yes,110,Driver,Female,49,Inner Regional Australia,Far West and Orana,Dubbo Regional,National or State Highway,No,No
2,120161097596,NSW,4,2016,Tuesday,16:40:00,Multiple,No,No,Yes,110,Passenger,Female,80,Inner Regional Australia,Far West and Orana,Dubbo Regional,National or State Highway,No,No
3,120161098282,NSW,4,2016,Sunday,14:00:00,Single,No,No,No,80,Passenger,Male,13,Inner Regional Australia,Riverina,Lockhart,Local Road,No,No
4,120161098913,NSW,4,2016,Saturday,07:30:00,Single,No,No,No,110,Driver,Male,21,Outer Regional Australia,Far West and Orana,Narromine,National or State Highway,No,No


In [4]:
fatal_crashes.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas 2021,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period
0,620172123,TAS,1,2017,Friday,16:10:00,Multiple,1,No,No,No,60,Inner Regional Australia,Hobart,Hobart,Sub-arterial Road,No,No
1,620172124,TAS,1,2017,Friday,19:00:00,Single,1,No,No,No,100,Outer Regional Australia,Launceston and North East,Northern Midlands,Arterial Road,No,No
2,620172125,TAS,2,2017,Monday,13:50:00,Multiple,1,No,Yes,No,100,Outer Regional Australia,Launceston and North East,Break O'Day,Arterial Road,No,No
3,620172126,TAS,3,2017,Thursday,11:29:00,Single,1,No,No,No,50,Inner Regional Australia,Launceston and North East,West Tamar,Local Road,No,No
4,620172127,TAS,3,2017,Saturday,13:20:00,Multiple,1,No,No,No,100,Remote Australia,West and North West,West Coast,Arterial Road,No,No


I want to explore the following questions using the data:
- How did fatalities vary based on the age and vehicle type?
- Which locations have the highest crash rates
- Is there any discrepancy in fatality rates between Male and Female drivers? How does has this changed over time?
- How do fatalities change based the type of road user and the speed limit? How does this vary over time, and between states?
- What time of day do fatal crashes occur the most? Is there a particlar day of the week that has the highest rate?

I constructed my warehouse schema in (filename). I will use this structure to separate the data we have loaded in, into dimension and fact tables.

First, I will merge the two dataframes into one on `Crash ID`.

First we will make the dim_time dimension table.

In [5]:
# Check for overlapping columns
overlap_cols = set(fatalities.columns).intersection(set(fatal_crashes.columns))
overlap_cols.remove('Crash ID')

print(overlap_cols)

{'Time', 'Heavy Rigid Truck Involvement', 'Speed Limit', 'Easter Period', 'State', 'Bus Involvement', 'Articulated Truck Involvement', 'Year', 'National LGA Name 2021', 'Dayweek', 'National Remoteness Areas 2021', 'Month', 'Christmas Period', 'National Road Type', 'SA4 Name 2021', 'Crash Type'}


In [6]:
# Merge the two dataframes on "Crash ID"
crashes_merged = pd.merge(fatalities, fatal_crashes, on="Crash ID", how="left", suffixes=('_Fatality', '_Crash'))

In [7]:
crashes_merged.columns

Index(['Crash ID', 'State_Fatality', 'Month_Fatality', 'Year_Fatality',
       'Dayweek_Fatality', 'Time_Fatality', 'Crash Type_Fatality',
       'Bus Involvement_Fatality', 'Heavy Rigid Truck Involvement_Fatality',
       'Articulated Truck Involvement_Fatality', 'Speed Limit_Fatality',
       'Road User', 'Gender', 'Age', 'National Remoteness Areas 2021_Fatality',
       'SA4 Name 2021_Fatality', 'National LGA Name 2021_Fatality',
       'National Road Type_Fatality', 'Christmas Period_Fatality',
       'Easter Period_Fatality', 'State_Crash', 'Month_Crash', 'Year_Crash',
       'Dayweek_Crash', 'Time_Crash', 'Crash Type_Crash', 'Number Fatalities',
       'Bus Involvement_Crash', 'Heavy Rigid Truck Involvement_Crash',
       'Articulated Truck Involvement_Crash', 'Speed Limit_Crash',
       'National Remoteness Areas 2021_Crash', 'SA4 Name 2021_Crash',
       'National LGA Name 2021_Crash', 'National Road Type_Crash',
       'Christmas Period_Crash', 'Easter Period_Crash'],
      dt

In [8]:
# Check if matching columns have identical values
print((crashes_merged['State_Fatality'] == crashes_merged['State_Crash']).all())

True


In [9]:
# Drop redundant columns
cols_to_drop = ['State_Crash', 'Month_Crash', 'Year_Crash', 'Dayweek_Crash', 'Time_Crash', 
                'National Remoteness Areas 2021_Crash', 'SA4 Name 2021_Crash', 
                'National LGA Name 2021_Crash', 'Speed Limit_Crash', 'Crash Type_Fatality',
                'National Road Type_Crash', 'Christmas Period_Crash', 'Easter Period_Crash']

crashes_merged.drop(cols_to_drop, axis=1, inplace=True)

In [10]:
crashes_merged.columns

Index(['Crash ID', 'State_Fatality', 'Month_Fatality', 'Year_Fatality',
       'Dayweek_Fatality', 'Time_Fatality', 'Bus Involvement_Fatality',
       'Heavy Rigid Truck Involvement_Fatality',
       'Articulated Truck Involvement_Fatality', 'Speed Limit_Fatality',
       'Road User', 'Gender', 'Age', 'National Remoteness Areas 2021_Fatality',
       'SA4 Name 2021_Fatality', 'National LGA Name 2021_Fatality',
       'National Road Type_Fatality', 'Christmas Period_Fatality',
       'Easter Period_Fatality', 'Crash Type_Crash', 'Number Fatalities',
       'Bus Involvement_Crash', 'Heavy Rigid Truck Involvement_Crash',
       'Articulated Truck Involvement_Crash'],
      dtype='object')

In [11]:
# Simplify col names
crashes_merged.rename(columns=lambda x: x.replace('_Fatality', ''), inplace=True)
crashes_merged.rename(columns=lambda x: x.replace('_Crash', ''), inplace=True)

In [12]:
crashes_merged.columns

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time',
       'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit', 'Road User', 'Gender',
       'Age', 'National Remoteness Areas 2021', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Crash Type', 'Number Fatalities', 'Bus Involvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement'],
      dtype='object')

In [13]:
def convert_time(time_str):
    """Convert time string, rounded down to the nearest hour.
    Args:
        time_str (str): Time string in the format 'HH:MM:SS'.
    Returns:
        int: Hour of the day (0-23).
    """
    try:
        hour = int(str(time_str).split(':')[0])
        if 0 <= hour <= 23:
            return hour
        else:
            return None
    except (ValueError, IndexError, AttributeError):
        return None

In [14]:
crashes_merged['Time']

0        15:29:00
1        16:40:00
2        16:40:00
3        14:00:00
4        07:30:00
           ...   
57080    16:00:00
57081    02:30:00
57082    13:20:00
57083    11:20:00
57084    12:20:00
Name: Time, Length: 57085, dtype: object

In [15]:
crashes_merged['Time'] = crashes_merged['Time'].apply(convert_time)

In [16]:
dimtime = crashes_merged[['Year', 'Month', 'Dayweek', 'Time']].drop_duplicates().reset_index(drop=True)

In [17]:
dimtime.head()

,Year,Month,Dayweek,Time
0,2016,4,Monday,15.0
1,2016,4,Tuesday,16.0
2,2016,4,Sunday,14.0
3,2016,4,Saturday,7.0
4,2016,4,Sunday,17.0


In [18]:
# checking for null values
dimtime.isnull().sum()

Year        0
Month       0
Dayweek     0
Time       36
dtype: int64

In [19]:
time_nullunknown_indexes = dimtime[
    (dimtime['Time'] == 'Unknown') | (dimtime['Time'].isnull())
].index
print(time_nullunknown_indexes)
dimtime.drop(time_nullunknown_indexes, inplace=True)

Index([ 9314, 14558, 14820, 14884, 16017, 17738, 17932, 18087, 19114, 19170,
       19521, 19878, 21287, 23035, 24494, 25421, 25528, 27217, 28913, 30777,
       31043, 33885, 33886, 33981, 33982, 34013, 34117, 34184, 34311, 34376,
       34408, 34652, 34660, 34661, 34740, 34791],
      dtype='int64')


In [20]:
# check each column for odd values
print(dimtime['Year'].unique())
print(dimtime['Month'].unique())
print(dimtime['Dayweek'].unique())
print(dimtime['Time'].unique())


[2016 2024 2017 2019 2015 2023 2018 2022 2021 2020 2025 2014 1989 1990
 1994 1997 2000 1991 1992 1999 1996 1998 1993 1995 2002 2003 2005 2006
 2001 2009 2004 2007 2010 2011 2008 2012 2013]
[ 4 11  8  3 12  1  2  7  5  9 10  6]
['Monday' 'Tuesday' 'Sunday' 'Saturday' 'Thursday' 'Friday' 'Wednesday']
[15. 16. 14.  7. 17. 13. 10.  8. 11.  1.  0.  5. 19. 21.  2. 20.  9.  6.
 22.  4. 23.  3. 12. 18.]


In [21]:
dimtime['time_id'] = dimtime.index + 1

dimtime.columns = ['year', 'month', 'dayweek', 'hour', 'time_id']
dimtime = dimtime[['time_id', 'year', 'month', 'dayweek', 'hour']]

In [22]:
dimtime.head()

,time_id,year,month,dayweek,hour
0,1,2016,4,Monday,15.0
1,2,2016,4,Tuesday,16.0
2,3,2016,4,Sunday,14.0
3,4,2016,4,Saturday,7.0
4,5,2016,4,Sunday,17.0


In [23]:
print(dimtime.info())
print(dimtime.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 34863 entries, 0 to 34898
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   time_id  34863 non-null  int64  
 1   year     34863 non-null  int64  
 2   month    34863 non-null  int64  
 3   dayweek  34863 non-null  object 
 4   hour     34863 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.6+ MB
None
            time_id          year         month          hour
count  34863.000000  34863.000000  34863.000000  34863.000000
mean   17440.671170   2005.212489      6.518114     12.444884
std    10072.431956     10.469093      3.474793      6.489781
min        1.000000   1989.000000      1.000000      0.000000
25%     8716.500000   1996.000000      3.000000      7.000000
50%    17437.000000   2004.000000      7.000000     13.000000
75%    26164.500000   2014.000000     10.000000     18.000000
max    34899.000000   2025.000000     12.000000     23.000000


Next I will construct the dim_location table.

In [24]:
dimlocation = crashes_merged[['State', 'National LGA Name 2021', 'SA4 Name 2021', 'National Remoteness Areas 2021']].drop_duplicates().reset_index(drop=True)
dimlocation.columns = ['state', 'location_name', 'region', 'location_type']

In [25]:
# check for null values
print(dimlocation.isnull().sum())

print(dimlocation['state'].unique())
print(dimlocation['region'].unique())
print(dimlocation['location_type'].unique())

state            0
location_name    0
region           0
location_type    0
dtype: int64
['NSW' 'QLD' 'VIC' 'WA' 'SA' 'TAS' 'NT' 'ACT']
['New England and North West' 'Far West and Orana' 'Riverina'
 'Newcastle and Lake Macquarie' 'Capital Region' 'Central West'
 'Southern Highlands and Shoalhaven' 'Central Coast'
 'Sydney - Northern Beaches' 'Murray' 'Hunter Valley exc Newcastle'
 'Sydney - South West' 'Richmond - Tweed' 'Coffs Harbour - Grafton'
 'Sydney - Parramatta' 'Sydney - Ryde' 'Sydney - Sutherland'
 'Sydney - Inner West' 'Sydney - Blacktown' 'Illawarra' 'Mid North Coast'
 'Sydney - Baulkham Hills and Hawkesbury' 'Sydney - Outer South West'
 'Sydney - North Sydney and Hornsby' 'Sydney - Inner South West'
 'Sydney - Eastern Suburbs' 'Sydney - City and Inner South'
 'Sydney - Outer West and Blue Mountains' 'Gold Coast'
 'Mackay - Isaac - Whitsunday' 'Toowoomba' 'Moreton Bay - North' 'Cairns'
 'Central Queensland' 'Queensland - Outback' 'Sunshine Coast'
 'Logan - Beaudesert' 'Ipswi

In [26]:
location_nullunknown_indexes = dimlocation[
    dimlocation['location_type'] == 'Unknown'
].index
print(location_nullunknown_indexes)
dimlocation.drop(location_nullunknown_indexes, inplace=True)

Index([297, 431, 433, 491, 624, 645, 708, 709, 793, 806, 831, 835, 836, 837], dtype='int64')


In [27]:
dimlocation['location_id'] = dimlocation.index + 1

# rename columns
dimlocation.columns = ['state', 'location_name', 'region', 'location_type', 'location_id']
dimlocation = dimlocation[['location_id', 'location_name', 'region', 'state', 'location_type']]

In [28]:
dimlocation.head()

,location_id,location_name,region,state,location_type
0,1,Tamworth Regional,New England and North West,NSW,Inner Regional Australia
1,2,Dubbo Regional,Far West and Orana,NSW,Inner Regional Australia
2,3,Lockhart,Riverina,NSW,Inner Regional Australia
3,4,Narromine,Far West and Orana,NSW,Outer Regional Australia
4,5,Newcastle,Newcastle and Lake Macquarie,NSW,Major Cities of Australia


Now for the dim_person table.

In [29]:
# function to convert age to age_group
def age_group(age):
    if age < 18:
        return  '0-17'
    elif age <= 24:
        return '18-24'
    elif age <= 29:
        return '25-29'
    elif age <= 34:
        return '30-34'
    elif age <= 39:
        return '35-39'
    elif age <= 44:
        return '40-44'
    elif age <= 49:
        return '45-49'
    elif age <= 54:
        return '50-54'
    elif age <= 59:
        return '55-59'
    elif age <= 64:
        return '60-64'
    elif age <= 69:
        return '65-69'
    else:
        return '70+'

In [30]:
# create age_group column
crashes_merged['age_group'] = crashes_merged['Age'].apply(age_group)

# create dimpersondetails dataframe
dimpersondetails = crashes_merged[['Gender', 'age_group', 'Road User']].drop_duplicates().reset_index(drop=True)
dimpersondetails.reset_index(drop=True, inplace=True)

In [31]:
print(dimpersondetails['Gender'].unique())
print(dimpersondetails['Road User'].unique())
print(dimpersondetails['age_group'].unique())

['Male' 'Female' 'Unknown']
['Driver' 'Passenger' 'Motorcycle rider' 'Pedal cyclist' 'Pedestrian'
 'Motorcycle pillion passenger' 'Unknown']
['70+' '45-49' '0-17' '18-24' '25-29' '50-54' '65-69' '30-34' '40-44'
 '60-64' '55-59' '35-39']


In [32]:
print(dimpersondetails[dimpersondetails['Gender'] == 'Unknown'])
print(dimpersondetails[dimpersondetails['Road User'] == 'Unknown'])

      Gender age_group         Road User
104  Unknown      0-17         Passenger
105  Unknown     35-39  Motorcycle rider
119  Unknown     55-59  Motorcycle rider
154  Unknown      0-17        Pedestrian
159  Unknown      0-17            Driver
164  Unknown       70+  Motorcycle rider
     Gender age_group Road User
111    Male     60-64   Unknown
112    Male      0-17   Unknown
113  Female     30-34   Unknown
114    Male     65-69   Unknown
121    Male     18-24   Unknown
123    Male     30-34   Unknown
127    Male       70+   Unknown
136    Male     25-29   Unknown
138  Female     18-24   Unknown
139    Male     40-44   Unknown
140    Male     35-39   Unknown
144  Female       70+   Unknown
146    Male     45-49   Unknown
149  Female     45-49   Unknown
150  Female      0-17   Unknown
151    Male     55-59   Unknown
153  Female     60-64   Unknown
156  Female     50-54   Unknown
157  Female     35-39   Unknown
161  Female     25-29   Unknown
163    Male     50-54   Unknown
166  Fema

In [33]:
person_nullunknown_indexes = dimpersondetails[
    (dimpersondetails['Gender'] == 'Unknown') | (dimpersondetails['Road User'] == 'Unknown')
].index
print(person_nullunknown_indexes)
dimpersondetails.drop(person_nullunknown_indexes, inplace=True)

Index([104, 105, 111, 112, 113, 114, 119, 121, 123, 127, 136, 138, 139, 140,
       144, 146, 149, 150, 151, 153, 154, 156, 157, 159, 161, 163, 164, 166,
       167],
      dtype='int64')


In [34]:
# add primary key
dimpersondetails['person_id'] = dimpersondetails.index + 1
dimpersondetails.columns = ['gender', 'age_group', 'road_user', 'person_id']

# reorder columns
dimpersondetails = dimpersondetails[['person_id', 'gender', 'age_group', 'road_user']]

In [35]:
dimcrashdetails = crashes_merged[['Crash Type', 'Speed Limit', 'National Road Type']].drop_duplicates().reset_index(drop=True)

In [36]:
dimcrashdetails.describe()

,Speed Limit
count,170.000000
mean,60.364706
std,37.111649
min,-9.000000
25%,30.000000
50%,60.000000
75%,90.000000
max,130.000000


In [37]:
print(dimcrashdetails[dimcrashdetails['Speed Limit'] <= 0])

    Crash Type  Speed Limit         National Road Type
76      Single           -9             Collector Road
78      Single           -9          Sub-arterial Road
81      Single           -9                    Unknown
83      Single           -9                Access road
86      Single           -9                 Local Road
97    Multiple           -9                    Unknown
108     Single           -9  National or State Highway
111   Multiple           -9          Sub-arterial Road
112   Multiple           -9                 Local Road
122   Multiple           -9  National or State Highway
129   Multiple           -9              Arterial Road
130     Single           -9              Arterial Road


In [38]:
crashes_merged[crashes_merged['Speed Limit'] == -9].count() / crashes_merged.count()

Crash ID                          0.024998
State                             0.024998
Month                             0.024998
Year                              0.024998
Dayweek                           0.024998
Time                              0.024788
Bus Involvement                   0.024998
Heavy Rigid Truck Involvement     0.024998
Articulated Truck Involvement     0.024998
Speed Limit                       0.024998
Road User                         0.024998
Gender                            0.024998
Age                               0.024998
National Remoteness Areas 2021    0.024998
SA4 Name 2021                     0.024998
National LGA Name 2021            0.024998
National Road Type                0.024998
Christmas Period                  0.024998
Easter Period                     0.024998
Crash Type                        0.024998
Number Fatalities                 0.024998
Bus Involvement                   0.024998
Heavy Rigid Truck Involvement     0.024998
Articulated

In [39]:
crash_nullunknown_indexes = dimcrashdetails[
    dimcrashdetails['Speed Limit'] == -9
].index

print(crash_nullunknown_indexes)

dimcrashdetails.drop(crash_nullunknown_indexes, inplace=True)

Index([76, 78, 81, 83, 86, 97, 108, 111, 112, 122, 129, 130], dtype='int64')


In [40]:
# add primary key
dimcrashdetails['crashdetails_id'] = dimcrashdetails.index + 1
dimcrashdetails.columns = ['crash_type', 'speed_limit', 'road_type', 'crashdetails_id']

# reorder columns
dimcrashdetails = dimcrashdetails[['crashdetails_id', 'crash_type', 'speed_limit', 'road_type']]

In [41]:
dimcrashdetails.head()

,crashdetails_id,crash_type,speed_limit,road_type
0,1,Multiple,100,Arterial Road
1,2,Multiple,110,National or State Highway
2,3,Single,80,Local Road
3,4,Single,110,National or State Highway
4,5,Single,80,Arterial Road


In [42]:
dimtime.head()

,time_id,year,month,dayweek,hour
0,1,2016,4,Monday,15.0
1,2,2016,4,Tuesday,16.0
2,3,2016,4,Sunday,14.0
3,4,2016,4,Saturday,7.0
4,5,2016,4,Sunday,17.0


In [43]:
dimlocation.head()

,location_id,location_name,region,state,location_type
0,1,Tamworth Regional,New England and North West,NSW,Inner Regional Australia
1,2,Dubbo Regional,Far West and Orana,NSW,Inner Regional Australia
2,3,Lockhart,Riverina,NSW,Inner Regional Australia
3,4,Narromine,Far West and Orana,NSW,Outer Regional Australia
4,5,Newcastle,Newcastle and Lake Macquarie,NSW,Major Cities of Australia


In [44]:
dimpersondetails.head()

,person_id,gender,age_group,road_user
0,1,Male,70+,Driver
1,2,Female,45-49,Driver
2,3,Female,70+,Passenger
3,4,Male,0-17,Passenger
4,5,Male,18-24,Driver


In [45]:
factfatality = crashes_merged.copy()

# Create fatality_id
factfatality['fatality_id'] = factfatality.index + 1

# Join with dim_person
factfatality = factfatality.merge(dimpersondetails, how='left', 
    left_on=['Gender', 'age_group', 'Road User'], 
    right_on=['gender', 'age_group', 'road_user'])

# Join with dim_location
factfatality = factfatality.merge(dimlocation, how='left', 
    left_on=['National LGA Name 2021', 'SA4 Name 2021', 'State', 'National Remoteness Areas 2021'],
    right_on=['location_name', 'region', 'state', 'location_type'])

# Join with dim_time
factfatality = factfatality.merge(dimtime, how='left',
    left_on=['Year', 'Month', 'Dayweek', 'Time'],
    right_on=['year', 'month', 'dayweek', 'hour'])

# Join with dim_crashdetails
factfatality = factfatality.merge(dimcrashdetails, how='left',
    left_on=['Crash Type', 'Speed Limit', 'National Road Type'],
    right_on=['crash_type', 'speed_limit', 'road_type'])


In [46]:
factfatality.head()

,Crash ID,State,Month,Year,Dayweek,Time,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,...,location_type,time_id,year,month,dayweek,hour,crashdetails_id,crash_type,speed_limit,road_type
0,120161098270,NSW,4,2016,Monday,15.0,No,No,No,100,...,Inner Regional Australia,1.0,2016.0,4.0,Monday,15.0,1.0,Multiple,100.0,Arterial Road
1,120161097596,NSW,4,2016,Tuesday,16.0,No,No,Yes,110,...,Inner Regional Australia,2.0,2016.0,4.0,Tuesday,16.0,2.0,Multiple,110.0,National or State Highway
2,120161097596,NSW,4,2016,Tuesday,16.0,No,No,Yes,110,...,Inner Regional Australia,2.0,2016.0,4.0,Tuesday,16.0,2.0,Multiple,110.0,National or State Highway
3,120161098282,NSW,4,2016,Sunday,14.0,No,No,No,80,...,Inner Regional Australia,3.0,2016.0,4.0,Sunday,14.0,3.0,Single,80.0,Local Road
4,120161098913,NSW,4,2016,Saturday,7.0,No,No,No,110,...,Outer Regional Australia,4.0,2016.0,4.0,Saturday,7.0,4.0,Single,110.0,National or State Highway


In [47]:
factfatality = factfatality[[
    'fatality_id', 'Crash ID', 'person_id', 'location_id', 'time_id', 'crashdetails_id'
]].rename(columns={'Crash ID': 'crash_id'})


In [48]:
factfatality.head()

,fatality_id,crash_id,person_id,location_id,time_id,crashdetails_id
0,1,120161098270,1.0,1.0,1.0,1.0
1,2,120161097596,2.0,2.0,2.0,2.0
2,3,120161097596,3.0,2.0,2.0,2.0
3,4,120161098282,4.0,3.0,3.0,3.0
4,5,120161098913,5.0,4.0,4.0,4.0


In [49]:
factcrash = crashes_merged.drop_duplicates(subset='Crash ID').copy()

factcrash = factcrash.merge(dimlocation, how='left',
    left_on=['National LGA Name 2021', 'SA4 Name 2021', 'State', 'National Remoteness Areas 2021'],
    right_on=['location_name', 'region', 'state', 'location_type'])

factcrash = factcrash.merge(dimtime, how='left',
    left_on=['Year', 'Month', 'Dayweek', 'Time'],
    right_on=['year', 'month', 'dayweek', 'hour'])

factcrash = factcrash.merge(dimcrashdetails, how='left',
    left_on=['Crash Type', 'Speed Limit', 'National Road Type'],
    right_on=['crash_type', 'speed_limit', 'road_type'])

factcrash = factcrash[[
    'Crash ID', 'location_id', 'time_id', 'crashdetails_id', 'Number Fatalities'
]].rename(columns={
    'Crash ID': 'crash_id',
    'Number Fatalities': 'number_fatalities'
})


In [50]:
factcrash.head()

,crash_id,location_id,time_id,crashdetails_id,number_fatalities
0,120161098270,1.0,1.0,1.0,2
1,120161097596,2.0,2.0,2.0,2
2,120161098282,3.0,3.0,3.0,1
3,120161098913,4.0,4.0,4.0,3
4,120161098283,5.0,5.0,5.0,1


In [52]:
# export tables to .csv files
dimtime.to_csv('data/dimtime.csv', index=False)
dimlocation.to_csv('data/dimlocation.csv', index=False)
dimpersondetails.to_csv('data/dimpersondetails.csv', index=False)
dimcrashdetails.to_csv('data/dimcrashdetails.csv', index=False)

factfatality.to_csv('data/factfatality.csv', index=False)
factcrash.to_csv('data/factcrash.csv', index=False)
